In [1]:
# prerequisite
# pip install -U imbalanced-learn

In [2]:
from pipelines.DataPipeline import DataPipeline
from transformers.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split

from transformers.Rebalancer import Rebalancer

from utils.DataHelper import DataHelper
from utils.Common import Config
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sklearn.naive_bayes import GaussianNB

import pandas as pd
import numpy as np


In [3]:
RAW_DATA_PATH = "../data/raw/KSI.csv"
df = pd.read_csv(RAW_DATA_PATH)

In [4]:
# fill missing values, adding new columns, extracting useful columns
pc = Preprocessing(df, Config.binary_columns, Config.cat_attribs, Config.num_attribs, Config.label)
new_df = pc.getFrame()

In [5]:
# seperate feature and label
X = new_df[Config.cat_attribs + Config.num_attribs+ Config.binary_columns]
Y = new_df[Config.label]

In [6]:
# pass feature to pipeline and convert it to numerical data
X = DataPipeline(Config.num_attribs,Config.cat_attribs).process(X)

In [7]:
X.isna().sum().sum()

0

In [8]:
Y.value_counts().tolist()

[14246, 2201]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=Config.test_size, stratify=Y)

In [10]:
from imblearn.over_sampling import SMOTE
smote_minority = SMOTE(n_jobs = -1, sampling_strategy = "minority")

X_train_sm, y_train_sm = smote_minority.fit_resample(X_train, y_train)


c:\Users\yvonn\anaconda3\lib\site-packages\imblearn\over_sampling\_smote\base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


In [11]:
# pip install xgboost

In [12]:
from xgboost import XGBClassifier
clf = XGBClassifier(learning_rate=0.01, n_estimators=600, objective='binary:logistic',
                 )
clf.fit(X_train_sm, y_train_sm)
clf.score(X_train_sm, y_train_sm)

0.9266848016848017

In [13]:
clf.score(X_test,y_test)

0.8708206686930091

In [14]:


# Create param grid
param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 2.5, 5],
        'subsample': [0.6, 1.0],
        'colsample_bytree': [0.6,1.0],
        'max_depth': [3, 5]
        }

clf = GridSearchCV(
    estimator=XGBClassifier(learning_rate=0.01, n_estimators=600, objective='binary:logistic',
                 ),
    scoring="accuracy",
    param_grid=param_grid,
    cv=10,
    refit=True,
    verbose=3
)

best_clf = clf.fit(X_train_sm, y_train_sm)

best_clf.best_score_

Fitting 10 folds for each of 72 candidates, totalling 720 fits
[CV 1/10] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.566 total time=   2.2s
[CV 2/10] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.730 total time=   2.2s
[CV 3/10] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.928 total time=   2.2s
[CV 4/10] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.933 total time=   2.3s
[CV 5/10] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.940 total time=   2.6s
[CV 6/10] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.929 total time=   2.4s
[CV 7/10] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.926 total time=   2.3s
[CV 8/10] END colsample_bytree=0.6, gamma=0.5, m

0.9049449396857657

In [15]:
best_model = best_clf.best_estimator_
best_model.score(X_test,y_test)

0.8699088145896656

In [16]:
best_clf.best_params_
# {'colsample_bytree': 1.0, 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.6}

{'colsample_bytree': 1.0,
 'gamma': 0.5,
 'max_depth': 5,
 'min_child_weight': 1,
 'subsample': 0.6}

In [21]:
import joblib

best_model = XGBClassifier(learning_rate=0.01, n_estimators=600, objective='binary:logistic',colsample_bytree= 1.0, gamma= 0.5, max_depth= 5, min_child_weight= 1, subsample= 0.6)
best_model.fit(X_train_sm, y_train_sm)
joblib.dump(best_model,'../models/best_model_xgboost.pkl')


['../models/best_model_xgboost.pkl']